In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, TFBertModel

# Step 1: Load and prepare the dataset
# Load dataset
df = pd.read_csv("./data.csv")  # Make sure the path is correct


f:\Mini Conda\envs\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [3]:
df['Sentiment'].value_counts()

Sentiment
neutral     3130
positive    1852
negative     860
Name: count, dtype: int64

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Step 1: Load data
df = pd.read_csv("./data.csv")  # Ensure correct path
print(df.head())

# Step 2: Preprocess the text and labels
df['Sentiment'] = df['Sentiment'].map({'positive': 1, 'negative': 0, 'neutral': 2})

# Step 3: Tokenization and BERT Embedding
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='tf', truncation=True, padding=True, max_length=128)
    outputs = bert_model(**inputs)
    return outputs.last_hidden_state[:, 0, :]  # Using the [CLS] token embedding

# Step 4: Define the Reversible Autoencoder
class ReversibleAutoencoder(tf.keras.Model):
    def __init__(self, embedding_size=768):
        super(ReversibleAutoencoder, self).__init__()
        self.encoder = models.Sequential([
            layers.Dense(512, activation='relu', input_shape=(embedding_size,)),
            layers.Dense(256, activation='relu')
        ])
        self.decoder = models.Sequential([
            layers.Dense(512, activation='relu', input_shape=(256,)),
            layers.Dense(embedding_size, activation='sigmoid')
        ])

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded, encoded

# Step 5: Define the Sentiment Classifier
class SentimentClassifier(tf.keras.Model):
    def __init__(self, embedding_size=768):
        super(SentimentClassifier, self).__init__()
        self.reversible_autoencoder = ReversibleAutoencoder(embedding_size)
        self.classifier = layers.Dense(3, activation='softmax')  # 3 classes: positive, negative, neutral

    def call(self, inputs):
        decoded, encoded = self.reversible_autoencoder(inputs)
        logits = self.classifier(encoded)
        return logits, decoded

# Step 6: Prepare Data for Training
X = df['Sentence'].apply(get_bert_embeddings).tolist()
X = tf.stack(X)  # Stack all sentence embeddings
y = tf.convert_to_tensor(df['Sentiment'].values)

# Step 7: Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X.numpy(), y.numpy(), test_size=0.2, random_state=42)

X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

# Step 8: Training the Model
model = SentimentClassifier(embedding_size=768)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Training Loop
model.fit(X_train, y_train, epochs=3, batch_size=16, validation_data=(X_test, y_test))

# Step 9: Evaluate the Model
y_pred = model.predict(X_test)[0]
predictions = tf.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy on Test Set: {accuracy * 100:.2f}%")


                                            Sentence Sentiment
0  The GeoSolutions technology will leverage Bene...  positive
1  $ESI on lows, down $1.50 to $2.50 BK a real po...  negative
2  For the last quarter of 2010 , Componenta 's n...  positive
3  According to the Finnish-Russian Chamber of Co...   neutral
4  The Swedish buyout firm has sold its remaining...   neutral


f:\Mini Conda\envs\env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are 